Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 5 Unweighted Average Ensembling

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
import os

In [2]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/dev.tsv', sep='\t')
models = ['dccuchile/bert-base-spanish-wwm-uncased', 'dccuchile/bert-base-spanish-wwm-cased', 'xlm-roberta-base', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased']
n_models = len(models)

epoch_string = '../15_epochs_small_model/saved_data_5'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())



In [3]:
# Retrieving all the predictions from the 
list_of_df = []

for model in models: 
    specific_model_row = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_result.tsv'
            
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)
    
    list_of_df.append(specific_model_row)

In [4]:
# TAKING THE UNWEIGHTED AVERAGE OF DATA

uw_avg_original_tweet_id_list = []
uw_avg_original_sentence_list = []
uw_avg_original_label_list = []
uw_avg_original_numbered_label_list = []

uw_avg_predicted_number_results = []
uw_avg_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['tweet_id']
    original_sentence = row['tweet_text']
    original_label = row['label']
    
    # transferring the labels over to final list
    uw_avg_original_tweet_id_list.append(original_tweet_id)
    uw_avg_original_sentence_list.append(original_sentence)
    uw_avg_original_label_list.append(original_label)
    
    specific_row_value = 0
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):
            # print(list_of_df[model_num][rnd_num])
            row = list_of_df[model_num][rnd_num].loc[(list_of_df[model_num][rnd_num]['tweet_id'] == original_tweet_id) & (list_of_df[model_num][rnd_num]['text'] == original_sentence)]
            
            prediction = labels_to_ids[row['label'].values[0]]
            specific_row_value += prediction

    specific_row_value = specific_row_value / 25
    specific_row_result = int(round(specific_row_value))
    uw_avg_predicted_results.append(ids_to_labels[specific_row_result])
    

In [5]:
# Calculating sklearn metrics

uw_avg_original_numbered_label_list = [labels_to_ids[label] for label in uw_avg_original_label_list]
uw_avg_predicted_number_results = [labels_to_ids[label] for label in uw_avg_predicted_results]

accuracy_result = accuracy_score(uw_avg_original_numbered_label_list, uw_avg_predicted_number_results)
f1_result = f1_score(uw_avg_original_numbered_label_list, uw_avg_predicted_number_results, labels=[0], average=None)[0]
precision_result = precision_score(uw_avg_original_numbered_label_list, uw_avg_predicted_number_results, labels=[0], average=None)[0]
recall_result = recall_score(uw_avg_original_numbered_label_list, uw_avg_predicted_number_results, labels=[0], average=None)[0]

confusion_matrix_result = confusion_matrix(uw_avg_original_numbered_label_list, uw_avg_predicted_number_results)
classification_result = classification_report(uw_avg_original_numbered_label_list, uw_avg_predicted_number_results, output_dict=True)

print("ACCURACY:", accuracy_result)
print("F1:", f1_result)
print("PRECISION:", precision_result)
print("RECALL", recall_result)

# Saving results to file
os.makedirs('../15_epochs_small_model/validating_statistics/uw_avg_ensemble', exist_ok=True)

with open('../15_epochs_small_model/validating_statistics/uw_avg_ensemble/uw_avg_ensemble_valid_stats.txt', 'w') as file:
        file.write("Accuracy: " + str(accuracy_result) + "\n")
        file.write("F1 Score (of self_reports): " + str(f1_result) + "\n")
        file.write("Precison Score (of self_reports): " + str(precision_result) + "\n")
        file.write("Recall result (of self_reports): " + str(recall_result) + "\n")


cm_df = pd.DataFrame(confusion_matrix_result)
cr_df = pd.DataFrame(classification_result).transpose()

cm_df.to_csv('../15_epochs_small_model/validating_statistics/uw_avg_ensemble/uw_avg_confusion_matrix.tsv', sep='\t')
cr_df.to_csv('../15_epochs_small_model/validating_statistics/uw_avg_ensemble/uw_avg_classification_report.tsv', sep='\t')




ACCURACY: 0.8405970149253731
F1: 0.7452229299363057
PRECISION: 0.6603951081843838
RECALL 0.8550548112058465


In [6]:
# Saving it as a dataframe
unformatted_uw_avg_prediction_data = pd.DataFrame(zip(uw_avg_original_tweet_id_list, uw_avg_original_sentence_list, uw_avg_original_label_list, uw_avg_predicted_results), columns=['tweet_id', 'text', 'Orig', 'label'])
formatted_uw_avg_prediction_data = unformatted_uw_avg_prediction_data.drop(columns = ['Orig'])

# Saving it as a tsv file
os.makedirs('../15_epochs_small_model/validating_statistics/', exist_ok=True)
unformatted_uw_avg_prediction_data.to_csv('../15_epochs_small_model/validating_statistics/uw_avg_ensemble/unformatted_uw_avg_data.tsv', sep='\t', index=False)
formatted_uw_avg_prediction_data.to_csv('../15_epochs_small_model/validating_statistics/uw_avg_ensemble/formatted_uw_avg_data.tsv', sep='\t', index=False)